In [2]:
import pandas as pd
import numpy as np
import re
import datetime as dt

# Importar bases
Delitos sexuales (ds), homicidios (homi) y vilencia intrafamiliar (VIF)

In [3]:
ds = pd.read_excel("ds.xlsx")
homi = pd.read_excel("homicidios.xlsx")
vif = pd.read_excel("vif.xlsx")

# Procesamiento VIF

- Remover el los registros sin cedula
- Contar el número de casos por víctima teniendo encuenta el id de denuncia
- Extraer en una base la primera o unica denuncia por víctima 
- Imputar datos en caso de registros faltantes si una persona tiene multiples registros

In [4]:
vif.shape
###cambiar los sin dato por NA
vif = vif.replace("Sin dato", np.nan)
vif = vif.replace("SIN DATO", np.nan)

In [5]:
## Remover el los registros sin cedula
cedulas_malas =[ "#0", "#0.0", "#1", "#1.0", "#999999.0", "#99999999", "#9999999999", "#43", "#1034", "#11111.0", "#11111111.0", "#1111", "#111111.0", "#1111111", "#11111111.0", "#1111111.0"]
vif = vif[~vif['num_documento'].isin(cedulas_malas)]
#vif2 = vif.query('num_documento != "SIN DATO"')
vif2 = vif.dropna(subset=["num_documento"])
vif2 = vif2.dropna(subset=["consecutivo_theta"])
vif2["numero_documento"]= vif2["num_documento"].str.extract(pat='(\d+)', expand=False).astype(int)
print(vif.shape)
print(vif2.shape)

(67878, 61)
(37084, 62)


In [6]:
## Contar el número de casos por víctima teniendo encuenta el id de denuncia
casos_vif = vif2.drop_duplicates(subset=['numero_documento', 'consecutivo_theta'], keep= 'last').groupby('numero_documento', as_index=False)['consecutivo_theta'].count()
casos_vif.rename(columns={"consecutivo_theta": "reincidencia"}, inplace=True)

In [7]:
casos_vif.groupby("reincidencia").count()

,numero_documento
reincidencia,
1,31914
2,2128
3,242
4,36
5,3
6,2
8,1


In [8]:
## Extraer en una base la primera o unica denuncia por víctima
primer_caso = vif2.sort_values('fc_denuncia').drop_duplicates(subset=['numero_documento'], keep= 'first')
primer_caso.columns

Index(['apellido_victima', 'consecutivo_theta', 'fc_denuncia', 'fecha_hecho',
       'dia', 'mes', 'anio_x', 'hora', 'fl_testigo', 'arma_medio',
       'barrio_residencia', 'codigo_barrio_residencia', 'comuna_residencia',
       'codigo_comuna_residencia', 'conducta_especial', 'estado_civil',
       'lugar', 'id_grupo_lugar', 'publico_lugar', 'grupo_lugar',
       'nivel_academico', 'ocupacion', 'ocupacion_vulnerable',
       'sede_receptora', 'id_grupo_sede_receptora', 'grupo_sede_receptora',
       'sexo', 'nombre_victima', 'num_documento', 'num_edad', 'tipo_violencia',
       'direccion_hecho', 'barrio', 'codigo_barrio_sisc', 'comuna',
       'codigo_comuna_sisc', 'geo_lat', 'geo_lon', 'plana_x', 'plana_y',
       'rango_edad_pisc', 'rango_edad', 'cd_stage_x', 'barrio_hecho',
       'codigo_barrio_hecho', 'comuna_hecho', 'codigo_comuna_hecho',
       'cuadrante', 'jornada', 'conducta', 'fc_actualizacion_x',
       'modalidad_1257', 'modalidad_294', 'base_origen', 'pais_nace',
      

In [9]:
primer_caso = primer_caso.merge(casos_vif, on='numero_documento', how='left')

In [10]:
print(primer_caso[["tipo_violencia", "ocupacion", "num_edad", "fl_testigo", "estado_civil", "arma_medio", "modalidad_1257", "modalidad_294","nivel_academico", "sexo", "relato"]].isna().sum())
print(primer_caso.shape)

tipo_violencia     34326
ocupacion           8147
num_edad               0
fl_testigo             0
estado_civil        1833
arma_medio         32181
modalidad_1257      1737
modalidad_294       1737
nivel_academico     3199
sexo                   0
relato              6799
dtype: int64
(34326, 63)


In [11]:
##- Imputar datos en caso de registros faltantes si una persona tiene multiples registros
#intentamos usar la segunda denuncia para la imputacion
casos_vif_segunda = pd.merge(vif2, casos_vif[casos_vif["reincidencia"]>1] , how ='inner', on =['numero_documento']).sort_values('fc_denuncia').groupby('numero_documento', as_index=False)["numero_documento","fc_denuncia", "ocupacion", "num_edad", "fl_testigo", "estado_civil", "arma_medio", "modalidad_1257", "modalidad_294","nivel_academico", "sexo", "relato"].nth(1)

/tmp/ipykernel_4095/172054366.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  casos_vif_segunda = pd.merge(vif2, casos_vif[casos_vif["reincidencia"]>1] , how ='inner', on =['numero_documento']).sort_values('fc_denuncia').groupby('numero_documento', as_index=False)["numero_documento","fc_denuncia", "ocupacion", "num_edad", "fl_testigo", "estado_civil", "arma_medio", "modalidad_1257", "modalidad_294","nivel_academico", "sexo", "relato"].nth(1)


In [12]:
#primer_caso[["numero_documento","fecha_denuncia","parentesco", "ocupacion", "edad", "testigo", "estado_civil", "arma_medio", "modalidad_1257", "modalidad_294","nivel_academico", "sexo"]]
primer_caso_new = pd.merge(primer_caso, casos_vif_segunda, how="left",  on =['numero_documento'])
primer_caso_new['arma_medio'] = np.where(primer_caso_new['arma_medio_x'].isna(), "Sin arma",primer_caso_new['arma_medio_x'])
primer_caso_new['estado_civil'] = np.where(primer_caso_new['estado_civil_x'].isna(), primer_caso_new['estado_civil_y'],primer_caso_new['estado_civil_x'])
primer_caso_new['sexo'] = np.where(primer_caso_new['sexo_x'].isna(), primer_caso_new['sexo_y'],primer_caso_new['sexo_x'])
primer_caso_new['modalidad_1257'] = np.where(primer_caso_new['modalidad_1257_x'].isna(), primer_caso_new['modalidad_1257_y'],primer_caso_new['modalidad_1257_x'])
primer_caso_new['ocupacion'] = np.where(primer_caso_new['ocupacion_x'].isna(), primer_caso_new['ocupacion_y'],primer_caso_new['ocupacion_x'])
primer_caso_new['nivel_academico'] = np.where(primer_caso_new['nivel_academico_x'].isna(), primer_caso_new['nivel_academico_y'],primer_caso_new['nivel_academico_x'])
primer_caso_new['modalidad_294'] = np.where(primer_caso_new['modalidad_294_x'].isna(), primer_caso_new['modalidad_294_y'],primer_caso_new['modalidad_294_x'])

print(primer_caso_new[[ "ocupacion", "num_edad_x", "fl_testigo_x", "estado_civil", "arma_medio", "modalidad_1257", "modalidad_294","nivel_academico", "sexo"]].isna().sum())

ocupacion          7882
num_edad_x            0
fl_testigo_x          0
estado_civil       1786
arma_medio            0
modalidad_1257     1621
modalidad_294      1621
nivel_academico    3109
sexo                  0
dtype: int64


In [14]:
##subbase con las variables viables a trabajar
primer_caso_new = primer_caso_new[["numero_documento","fc_denuncia_x","ocupacion","num_edad_x", "fl_testigo_x", "arma_medio", "modalidad_1257", "modalidad_294","nivel_academico", "sexo", "reincidencia", "relato_x"]]
primer_caso_new.rename(columns={"fc_denuncia_x": "fecha_denuncia", "num_edad_x": "edad", "fl_testigo_x": "testigo", "relato_x": "relato"}, inplace=True)
print(primer_caso_new.columns)

Index(['numero_documento', 'fecha_denuncia', 'ocupacion', 'edad', 'testigo',
       'arma_medio', 'modalidad_1257', 'modalidad_294', 'nivel_academico',
       'sexo', 'reincidencia', 'relato'],
      dtype='object')


# Procesamiento ds y homicidio
- Remover el los registros sin cedula
- Cambiar el formato de la varible "numero_documento"

In [15]:
primer_caso_new["fecha_denuncia"] =pd.to_datetime(primer_caso_new["fecha_denuncia"], format= '%Y-%m-%d')

In [16]:
homi = homi.dropna(subset=['numero_documento'])
homi["numero_documento"]= homi["numero_documento"].str.extract(pat='(\d+)', expand=False).astype(int)
ds = ds.dropna(subset=['numero_documento'])
ds["numero_documento"]= ds["numero_documento"].str.extract(pat='(\d+)', expand=False).astype(int)


# Union de las bases de homicidio y ds con vif
- Union de ds con vif teniendo en cuenta que solo contar los registros que tienen 7 dias o más de diferencia y que la base de vif puede tener id repetidos.
- Union de homicidio con vif

In [17]:
#union con ds
print(primer_caso_new.shape)
primer_caso = pd.merge(primer_caso_new, ds, how="left",  on =['numero_documento'])
primer_caso["incluir_ds"] = np.where(abs(((pd.to_datetime(primer_caso['fecha_denuncia_y'])- primer_caso['fecha_denuncia_x']).dt.days)<=7)| primer_caso['fecha_denuncia_y'].isna() , 0, 1)
primer_caso = primer_caso.groupby("numero_documento", as_index=False)["incluir_ds"].sum()
primer_caso_new = pd.merge(primer_caso_new, primer_caso, how="left",  on =['numero_documento'])
primer_caso_new.shape

(34326, 12)


(34326, 13)

In [18]:
primer_caso.groupby("incluir_ds")["incluir_ds"].count()

incluir_ds
0    34094
1      224
2        8
Name: incluir_ds, dtype: int64

In [19]:
#union con homi
primer_caso_new = pd.merge(primer_caso_new, homi[["numero_documento", "Index"]], how="left",  on =['numero_documento'])
primer_caso_new.shape

(34326, 14)

In [20]:
primer_caso_new.columns

Index(['numero_documento', 'fecha_denuncia', 'ocupacion', 'edad', 'testigo',
       'arma_medio', 'modalidad_1257', 'modalidad_294', 'nivel_academico',
       'sexo', 'reincidencia', 'relato', 'incluir_ds', 'Index'],
      dtype='object')

In [21]:
#base final
primer_caso_new["reincidente"] = np.where((primer_caso_new['incluir_ds'] <=0) & (primer_caso_new['reincidencia']<=1) & (primer_caso_new['Index'].isna()), 0, 1)

In [22]:
primer_caso_new.groupby("reincidente")["reincidente"].count()

reincidente
0    31725
1     2601
Name: reincidente, dtype: int64

In [23]:
#Exportar base
primer_caso_new.to_excel("union_vif_ds.xlsx")